# Data operations
* Disk vs memory
* Filtering
* Selection of columns
    * Keep/Drop/Rename
    * Sorting by values   
* Merging    
* Operations on columns
    * If/then logic 
    * map
    * String processing
    * Date functionality 
* Missing data    
* GroupBy
    * Aggregation
    * Transformation

## Disk vs memory

pandas operates exclusively in memory, where a SAS data set exists on disk. This means that the size of data able to be loaded in pandas is limited by your machine’s memory, but also that the operations on that data may be faster.

If out of core processing is needed, one possibility is the dask.dataframe library (currently in development) which provides a subset of pandas functionality for an on-disk DataFrame

## Read source

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
## Import data source
dict_source={'AE':'ae.csv','DM':'dm.csv','DS':'ds.csv','EX':'ex.csv','LB':'lb.csv',}
for k,v in dict_source.items():
    globals()['df_'+k]=pd.read_csv('./data/data_operation_source/'+v)                              

In [ ]:
#print
df_LB.head()

In [ ]:
# dtypes
df_LB.dtypes

## Filtering

In [ ]:
# Filtering
df_BILI=df_LB[df_LB['LBTESTCD']=='BILI']
df_BILI=df_BILI[df_BILI['LBORRES']>1]
df_BILI

## Selection of columns
### Keep

In [ ]:
# keep USUBJID LBTESTCD LBTEST LBORRES LBORNRLO LBORNRHI
df_BILI=df_BILI[['USUBJID','LBTESTCD','LBTEST','LBORRES','LBORNRLO','LBORNRHI']]
df_BILI

### Drop

In [ ]:
# drop LBTEST
df_BILI_drop=df_BILI.drop(columns=['LBTEST'])
df_BILI_drop

### Rename

In [ ]:
# rename 'LBORRES':'BILI'
df_BILI_rename=df_BILI_drop.rename(columns={'LBORRES':'BILI'})
df_BILI_rename

### Sorting by values

In [ ]:
# Sorting by USUBJID BILI
df_BILI_sort=df_BILI_rename.sort_values(['USUBJID','BILI'])
df_BILI_sort.head()

## Merging

In [ ]:
# left join SEX,BRTHDTC    
df_BILI_DM=pd.merge(df_BILI_sort,
                    df_DM[['USUBJID','SEX','RACE','BRTHDTC']],
                    how='left',
                    on=['USUBJID'])
df_BILI_DM

## Operations on columns

### If/then logic

In [ ]:
# df_BILI_flag
df_BILI_flag=df_BILI_DM.copy()
for index, row in df_BILI_flag.iterrows():
    if row['BILI']>1.5:
        df_BILI_flag.at[index,'flag']='high'
    elif row['BILI']<1.2:
        df_BILI_flag.at[index,'flag']='low' 
    else:
        df_BILI_flag.at[index,'flag']='normal'
df_BILI_flag[df_BILI_flag['flag']=='low'].head()

### map

In [ ]:
# drop_duplicates['LBTEST']
df_LB.drop_duplicates(['LBTEST'])['LBTESTCD']

In [ ]:
# dict_map
dict_map=dict(zip(df_LB.drop_duplicates(['LBTEST'])['LBTESTCD'],
                  df_LB.drop_duplicates(['LBTEST'])['LBTEST']))
dict_map

In [ ]:
# map LBTEST_DECODE
df_LBTEST=df_LB.copy()
df_LBTEST=df_LBTEST[['USUBJID','LBTESTCD']]
df_LBTEST['LBTEST_DECODE']=df_LBTEST['LBTESTCD'].map(dict_map)
df_LBTEST

### String processing

In [ ]:
# change the string column in df_LBTEST
# df_str_proc: 
df_str_proc=df_LBTEST.copy()
# string combine: USUBJID+LBTEST_DECODE
df_str_proc['col1']=df_str_proc['USUBJID']+'+'+df_str_proc['LBTEST_DECODE']
# upcase
df_str_proc['col2']=df_str_proc['col1'].str.upper()
# Extracting nth word
df_str_proc['col3']=df_str_proc['col2'].str.split("+", expand=True)[0]
# Extracting substring by position
df_str_proc['col4']=df_str_proc['col3'].str[0:1]
df_str_proc

### Date functionality

In [ ]:
# change the date column in df_BILI_DM
# df_date_func
df_date_func=df_BILI_DM.copy()
# BRTHDTC_dt pd.to_datetime()
df_date_func['BRTHDTC_dt']=pd.to_datetime(df_date_func['BRTHDTC'], errors='coerce')
# today
df_date_func['today']=pd.Timestamp("2022-08-18")
# diff/dt.days
df_date_func['diff']=df_date_func['today']-df_date_func['BRTHDTC_dt']
df_date_func['diff']=df_date_func['diff'].dt.days
# today_Y/today_M   .dt.year/.dt.month
df_date_func['today_Y']=df_date_func['today'].dt.year
df_date_func['today_M']=df_date_func['today'].dt.month

df_date_func

## Missing data
Both pandas and SAS have a representation for missing data.

pandas represents missing data with the special float value NaN (not a number). Many of the semantics are the same; 

## GroupBy

### Aggregation

In [ ]:
# grp_BILI groupby(['SEX','RACE'])['BILI'].sum()
grp_BILI=df_BILI_flag.groupby(['SEX','RACE'])['BILI'].sum()
grp_BILI

### Transformation

In [ ]:
# melt
df_DM_melt=pd.melt(df_DM,
                   id_vars=['USUBJID'], 
                   value_vars=['BRTHDTC','AGE','SEX','RACE'],
                   var_name='col_cat', 
                   value_name='col_value')   
df_DM_melt.sort_values('USUBJID').head(10)